# SETUP PROJECT 1

Importeer pandas voor DataFrames en pyodbc voor database-connecties:

In [10]:
import sqlite3
import pandas as pd
import pyodbc

aenc_conn = sqlite3.connect("../../../Data/Raw/Sqlite/aenc.sqlite")
adventureworks_conn = sqlite3.connect("../../../Data/Raw/Sqlite/AdventureWorks.sqlite")

Database Connectie:

In [11]:
DB_SDM = {"servername": r"143.177.235.181", "database": "DEDSp1", "username": "sa", "password": "Str0ngP@ssword"}
DB_NorthWind = {"servername": r"143.177.235.181", "database": "NorthWind", "username": "sa", "password": "Str0ngP@ssword"}
export_conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={DB_SDM['servername']};"
    f"DATABASE={DB_SDM['database']};"
    f"UID={DB_SDM['username']};"
    f"PWD={DB_SDM['password']};"
    f"Encrypt=yes;TrustServerCertificate=yes;"
)

NorthWind_conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={DB_NorthWind['servername']};"
    f"DATABASE={DB_NorthWind['database']};"
    f"UID={DB_NorthWind['username']};"
    f"PWD={DB_NorthWind['password']};"
    f"Encrypt=yes;TrustServerCertificate=yes;"
)

DataFrames maken voor tables:

In [12]:
def create_dataframes_sql(connection, db_type):
    dictionary : dict = {}
    query : str = ""
    key : str = ""

    if db_type == "sqlite":
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        key = "name"
    elif db_type == "ssms":
        query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
        key = "TABLE_NAME"

    table_names = pd.read_sql(query, connection)

    for table in table_names[key].tolist():
        try:
            dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
        except Exception as e:
            print(f"Error reading {table}: {e}")
            continue

    return dictionary


aenc = create_dataframes_sql(aenc_conn, "sqlite")
adventureworks = create_dataframes_sql(adventureworks_conn, "sqlite")
northwind = create_dataframes_sql(NorthWind_conn, "ssms")

#Errors en warnings zijn niet erg hier als het totaal 1 error en 4 UserWarnings zijn.

Error reading sysdiagrams: Could not decode to UTF-8 column 'definition' with text '������'


C:\Users\saadz\AppData\Local\Temp\ipykernel_15716\1853655394.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_names = pd.read_sql(query, connection)
C:\Users\saadz\AppData\Local\Temp\ipykernel_15716\1853655394.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
C:\Users\saadz\AppData\Local\Temp\ipykernel_15716\1853655394.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
C:

Dictionaries met DataFrames mergen naar één Dictionary:

In [15]:
sdm = aenc | adventureworks | northwind #Alle drie mergen in één Dictionary met alle DataFrames

sdm['Department'] = sdm['Department'].rename(columns={"dept_id": "DepartmentID", "dept_name": "DepartmentName"})
sdm['HumanResources_Department'] = sdm["HumanResources_Department"].rename(columns={"Name": "DepartmentName"})
sdm['Department'] = pd.concat([sdm["Department"], sdm["HumanResources_Department"]], ignore_index=True)
del sdm['HumanResources_Department']

sdm['Employee'] = sdm['Employee'].rename(columns={'emp_id': 'EmployeeID', 
                                                  'emp_fname': 'FirstName', 
                                                  'emp_lname': 'LastName', 
                                                  'city': 'City', 
                                                  'state': 'Region', 
                                                  'zip_code': 'PostalCode', 
                                                  'phone': 'HomePhone', 
                                                  'start_date': 'HireDate', 
                                                  'birth_date': 'BirthDate'})
sdm['Employees'] = pd.concat([sdm['Employees'], sdm['Employee']], ignore_index=True)
del sdm['Employee']



#Hier komt veranderen van Employee en Department zodat het overeenkomt met Source Data Model

#dict_order = [] #Hierin komt dictionary order te staan.
                 #Dit moet omdat de volgorde van welke tables worden gestuurd naar database belangrijk is.

#sdm = {k: sdm[k] for k in dict_order if k in sdm} #Verandert de volgorde van sdm-Dictionary.

print(len(list(sdm.keys())))

34


SDM in SSMS vullen:

In [ ]:
export_cursor = export_conn.cursor()

for table_name, df in sdm.items():
    try:
        for index, row in df.iterrows():
            columns = df.columns.tolist()

            values = []
            for col in columns:
                value = row[col]

                if pd.isna(value):
                    values.append("NULL")

                elif isinstance(value, str):
                    values.append(f"'{value.replace("'", "''")}'")

                else:
                    values.append(str(value))

            column_names = ", ".join(columns)
            value_string = ", ".join(values)
            query = f"INSERT INTO {table_name} ({column_names}) VALUES ({value_string})"

            export_cursor.execute(query)

        export_conn.commit()
        print(f"Table {table_name} complete!")
    except pyodbc.Error as e:
        print(f"Error in table: {table_name}")
        print(f"Query: {query}")
        print(f"Error message: {e}")
        print("-" * 80)

export_cursor.close()